In [27]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [28]:
DAYS_PREDICT = 15

In [29]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_dot = yf.download("DOT-USD", data_init, date_today)

df_dot.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-08-20,2.787857,3.077784,2.692896,2.900080,2.900080,48819867
2020-08-21,2.896923,3.090093,2.730919,2.875028,2.875028,49313137
2020-08-22,2.878570,4.510613,2.816762,4.484690,4.484690,485801096
2020-08-23,4.487058,4.487058,3.716271,3.967066,3.967066,320749483
2020-08-24,3.987413,4.783158,3.812670,4.602614,4.602614,407690171


In [30]:
df_dot.reset_index(inplace=True)
df_dot.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
753,2022-09-12,7.698387,8.033982,7.587331,7.661845,7.661845,479107652
754,2022-09-13,7.662024,7.918829,7.050818,7.134012,7.134012,478905956
755,2022-09-14,7.075026,7.283329,7.050818,7.203162,7.203162,318044357
756,2022-09-15,7.203431,7.219177,6.893924,6.926550,6.926550,372919866
757,2022-09-16,6.926949,6.968990,6.737572,6.896843,6.896843,303068671


In [31]:
df = df_dot[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [32]:
df.tail()

,ds,y
753,2022-09-12,7.661845
754,2022-09-13,7.134012
755,2022-09-14,7.203162
756,2022-09-15,6.926550
757,2022-09-16,6.896843


In [33]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [34]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [35]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:34:22 - cmdstanpy - INFO - Chain [1] start processing
00:34:22 - cmdstanpy - INFO - Chain [1] done processing


In [36]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
768,2022-09-27
769,2022-09-28
770,2022-09-29
771,2022-09-30
772,2022-10-01


In [37]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2020-08-20,5.527889,0.811344,6.833380,5.527889,5.527889,-0.313256,-0.313256,-0.313256,0.001253,0.001253,0.001253,-0.314508,-0.314508,-0.314508,0.0,0.0,0.0,3.796246
1,2020-08-21,5.496743,0.690053,6.930855,5.496743,5.496743,-0.299903,-0.299903,-0.299903,0.001885,0.001885,0.001885,-0.301788,-0.301788,-0.301788,0.0,0.0,0.0,3.848255
2,2020-08-22,5.465598,0.932127,7.201390,5.465598,5.465598,-0.285560,-0.285560,-0.285560,0.003400,0.003400,0.003400,-0.288961,-0.288961,-0.288961,0.0,0.0,0.0,3.904839
3,2020-08-23,5.434453,0.788349,7.098264,5.434453,5.434453,-0.272705,-0.272705,-0.272705,0.003400,0.003400,0.003400,-0.276105,-0.276105,-0.276105,0.0,0.0,0.0,3.952452
4,2020-08-24,5.403307,0.878797,7.333075,5.403307,5.403307,-0.268260,-0.268260,-0.268260,-0.004956,-0.004956,-0.004956,-0.263304,-0.263304,-0.263304,0.0,0.0,0.0,3.953816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
768,2022-09-27,10.633872,6.168862,12.518052,10.571974,10.685581,-0.133998,-0.133998,-0.133998,-0.004104,-0.004104,-0.004104,-0.129894,-0.129894,-0.129894,0.0,0.0,0.0,9.208952
769,2022-09-28,10.612528,6.048951,12.597768,10.529653,10.684618,-0.127051,-0.127051,-0.127051,-0.000878,-0.000878,-0.000878,-0.126173,-0.126173,-0.126173,0.0,0.0,0.0,9.264195
770,2022-09-29,10.591183,6.025188,12.381622,10.496004,10.684674,-0.120125,-0.120125,-0.120125,0.001253,0.001253,0.001253,-0.121378,-0.121378,-0.121378,0.0,0.0,0.0,9.318913
771,2022-09-30,10.569839,5.991679,12.499685,10.461685,10.691474,-0.113547,-0.113547,-0.113547,0.001885,0.001885,0.001885,-0.115432,-0.115432,-0.115432,0.0,0.0,0.0,9.369662


In [38]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
768,2022-09-27,6.168862,12.518052,9.208952
769,2022-09-28,6.048951,12.597768,9.264195
770,2022-09-29,6.025188,12.381622,9.318913
771,2022-09-30,5.991679,12.499685,9.369662
772,2022-10-01,6.168949,12.847004,9.442303


In [39]:
# Salvar modelo
with open('model_dot_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)